In [3]:
import os
import logging
import multiprocessing
import qiime2
import pandas as pd
from qiime2.plugins import feature_table, \
    dada2, \
    demux, \
    metadata, \
    alignment, \
    phylogeny, \
    diversity, \
    emperor, \
    feature_classifier, \
    taxa

In [124]:
def merge_run_tables(table1_artifact_path, table2_artifact_path):
    table1 = load_data_artifact(table1_artifact_path)
    table2 = load_data_artifact(table2_artifact_path)
    dada2_filtered_table = feature_table.actions.merge(table1=table1, table2=table2).merged_table
    return dada2_filtered_table


def merge_run_repseqs(repseqs1_artifact_path, repseqs2_artifact_path):
    repseqs1 = load_data_artifact(repseqs1_artifact_path)
    repseqs2 = load_data_artifact(repseqs2_artifact_path)
    dada2_filtered_rep_seqs = feature_table.actions.merge_seq_data(repseqs1, repseqs2).merged_data
    return dada2_filtered_rep_seqs


def load_data_artifact(filepath):
    """
    :param filepath: path to qiime2 artifact created with helper_functions.create_sampledata_artifact
    :return: qiime2 object containing all information on sequence data
    """
    data_artifact = qiime2.Artifact.load(filepath)
    return data_artifact


def load_sample_metadata(filepath):
    """
    :param filepath: path to the sample metadata file
    :return: qiime2 metadata object
    """
    metadata_object = qiime2.Metadata.load(filepath)
    return metadata_object


def visualize_metadata(base_dir, metadata_object):
    """
    :param base_dir: Main working directory filepath
    :param metadata_object: qiime2 metadata object
    :return: qiime2 metadata visualization object
    """
    # Path setup
    export_path = os.path.join(base_dir, 'sample-metadata-tabulate')

    # Prepare and save metadata visualization
    metadata_viz = metadata.visualizers.tabulate(metadata_object)
    metadata_viz.visualization.save(export_path)
    logging.info('Saved {} successfully'.format(export_path))

    return metadata_viz


def visualize_demux(base_dir, data_artifact):
    """
    :param base_dir: Main working directory filepath
    :param data_artifact: qiime2 data artifact object
    :return: qiime2 demux visualization object
    """
    logging.info('Running demux visualizer...')

    # Path setup
    export_path = os.path.join(base_dir, 'demux_summary.qzv')

    # Prepare and save demux summary visualization
    demux_viz = demux.visualizers.summarize(data=data_artifact)
    demux_viz.visualization.save(export_path)
    logging.info('Saved {} successfully'.format(export_path))

    return demux_viz

# RUN 1 PARAMS
def dada2_qc(base_dir, demultiplexed_seqs,
             trim_left_f=22, trim_left_r=7, trunc_len_f=0, trunc_len_r=220, max_ee=3,
             chimera_method='consensus', cpu_count=None):
    """
    :param base_dir: Main working directory filepath
    :param demultiplexed_seqs:
    :param trim_left_f:
    :param trim_left_r:
    :param trunc_len_f:
    :param trunc_len_r:
    :param chimera_method:
    :param cpu_count:
    :return: qiime2/dada2 filtered table and representative sequences objects
    """
    logging.info('Running dada2 for quality control of reads...')

    # Grab all CPUs if parameter is not specified
    if cpu_count is None:
        cpu_count = multiprocessing.cpu_count()
        logging.info('Set CPU count to {}'.format(cpu_count))

    # Run dada2
    (dada2_filtered_table, dada2_filtered_rep_seqs) = dada2.methods.denoise_paired(
        demultiplexed_seqs=demultiplexed_seqs,
        trim_left_f=trim_left_f,
        trim_left_r=trim_left_r,
        trunc_len_f=trunc_len_f,
        trunc_len_r=trunc_len_r,
        max_ee=max_ee,
        chimera_method=chimera_method,
        n_threads=cpu_count)

    # Save artifacts
    dada2_filtered_table.save(os.path.join(base_dir, 'table-dada2.qza'))
    dada2_filtered_rep_seqs.save(os.path.join(base_dir, 'rep-seqs-dada2.qza'))
    logging.info('Completed running dada2')

    return dada2_filtered_table, dada2_filtered_rep_seqs


def visualize_dada2(base_dir, dada2_filtered_table, dada2_filtered_rep_seqs, metadata_object):
    """
    :param base_dir: Main working directory filepath
    :param dada2_filtered_table: 
    :param dada2_filtered_rep_seqs: 
    :param metadata_object: 
    :return: qiime2 feature table summary object
    """
    # Prepare feature table
    feature_table_summary = feature_table.visualizers.summarize(table=dada2_filtered_table,
                                                                sample_metadata=metadata_object)

    # Prepare sequence table
    feature_table_seqs = feature_table.visualizers.tabulate_seqs(data=dada2_filtered_rep_seqs)

    # Save visualizations
    feature_table_summary.visualization.save(os.path.join(base_dir, 'table-dada2-summary.qzv'))
    feature_table_seqs.visualization.save(os.path.join(base_dir, 'rep-seqs-summary.qzv'))
    logging.info('Saved dada2 visualizations successfully.')

    return feature_table_summary


def seq_alignment_mask(base_dir, dada2_filtered_rep_seqs, cpu_count=None):
    """
    :param base_dir: Main working directory filepath
    :param dada2_filtered_rep_seqs: 
    :param cpu_count: number of CPUs to use for analysis
    :return: qiime2 sequence mask and sequence alignment objects
    """
    # Threading setup
    if cpu_count is None:
        cpu_count = multiprocessing.cpu_count()
        logging.info('Set CPU count to {}'.format(cpu_count))

    # Path setup
    aligned_export_path = os.path.join(base_dir, 'aligned-rep-seqs.qza')
    mask_export_path = os.path.join(base_dir, 'masked-aligned-rep-seqs.qza')

    # Perform and save sequence alignment
    seq_alignment = alignment.methods.mafft(sequences=dada2_filtered_rep_seqs, n_threads=cpu_count)
    seq_alignment.alignment.save(aligned_export_path)
    logging.info('Saved {} successfully'.format(aligned_export_path))

    # Perform and save alignment mask
    seq_mask = alignment.methods.mask(alignment=seq_alignment.alignment)
    seq_mask.masked_alignment.save(mask_export_path)
    logging.info('Saved {} successfully'.format(mask_export_path))

    return seq_mask, seq_alignment


def phylo_tree(base_dir, seq_mask):
    """
    :param base_dir: Main working directory filepath
    :param seq_mask: 
    :return: qiime2 unrooted and rooted tree objects
    """
    # Path setup
    unrooted_export_path = os.path.join(base_dir, 'unrooted-tree.qza')
    rooted_export_path = os.path.join(base_dir, 'rooted-tree.qza')

    # Run and save unrooted tree
    phylo_unrooted_tree = phylogeny.methods.fasttree(alignment=seq_mask.masked_alignment)
    phylo_unrooted_tree.tree.save(unrooted_export_path)
    logging.info('Saved {} successfully'.format(unrooted_export_path))

    # Run and save rooted tree
    phylo_rooted_tree = phylogeny.methods.midpoint_root(tree=phylo_unrooted_tree.tree)
    phylo_rooted_tree.rooted_tree.save(rooted_export_path)
    logging.info('Saved {} successfully'.format(rooted_export_path))

    return phylo_unrooted_tree, phylo_rooted_tree


def export_newick(base_dir, tree):
    """
    :param base_dir: Main working directory filepath
    :param tree: 
    :return: path to tree file in newick format exported from the tree object
    """
    # Path setup
    export_path = os.path.join(base_dir, 'newick.tree')

    # Export data
    tree.rooted_tree.export_data(export_path)
    logging.info('Exported tree file in newick format from the follwing artifact: {}'.format(tree))
    return export_path


def load_classifier_artifact(classifier_artifact_path):
    """
    :param classifier_artifact_path: 
    :return: qiime2 classifier object
    """
    # Load existing artifact
    classifier = qiime2.Artifact.load(classifier_artifact_path)
    logging.info('Loaded classifier from {}'.format(classifier_artifact_path))
    return classifier


def calculate_maximum_depth(dada2_table):
    """
    :param dada2_table:
    :return: maximum depth retrieved from the qiime2/dada2 table object
    """
    # Read in dada2 table
    df = dada2_table.view(pd.DataFrame)
    df = df.transpose()
    value_dict = {}

    # Calculate the sum of each column (sample)
    for column in df:
        value_dict[column] = df[column].sum()

    max_depth = max(value_dict.values())
    return max_depth


def alpha_rarefaction_visualization(base_dir, dada2_filtered_table, max_depth=None):
    """
    :param base_dir: Main working directory filepath
    :param dada2_filtered_table: 
    :param max_depth: 
    :return: qiime2 alpha rarefaction visualization object
    """
    # Path setup
    alpha_rarefaction_export_path = os.path.join(base_dir, 'alpha-rarefaction.qzv')

    # Max depth calculation. This (arbitrarily) sets the maximum depth to 80% of the highest value found.
    if max_depth is None:
        max_depth = int(calculate_maximum_depth(dada2_filtered_table) * 0.8)

    # Produce rarefaction curve
    alpha_rarefaction_viz = diversity.visualizers.alpha_rarefaction(table=dada2_filtered_table,
                                                                    max_depth=max_depth)

    # Save
    alpha_rarefaction_viz.visualization.save(alpha_rarefaction_export_path)
    logging.info('Saved {} successfully'.format(alpha_rarefaction_export_path))

    return alpha_rarefaction_viz


def classify_taxonomy(base_dir, dada2_filtered_rep_seqs, classifier, cpu_count=None):
    """
    :param base_dir: Main working directory filepath
    :param dada2_filtered_rep_seqs: 
    :param classifier: 
    :return: qiime2 post-classification taxonomy object
    """
    # Path setup
    export_path = os.path.join(base_dir, 'taxonomy.qza')

    # Threading setup
    if cpu_count is None:
        cpu_count = multiprocessing.cpu_count()
        logging.info('Set CPU count to {}'.format(cpu_count))

    # Classify reads
    taxonomy_analysis = feature_classifier.methods.classify_sklearn(reads=dada2_filtered_rep_seqs,
                                                                    classifier=classifier,
                                                                    n_jobs=cpu_count)
    # Save the resulting artifact
    taxonomy_analysis.classification.save(export_path)
    logging.info('Saved {} successfully'.format(export_path))

    return taxonomy_analysis


def visualize_taxonomy(base_dir, metadata_object, taxonomy_analysis, dada2_filtered_table):
    """
    :param base_dir: Main working directory filepath
    :param metadata_object: 
    :param taxonomy_analysis: 
    :param dada2_filtered_table: 
    :return: qiime2 taxonomy metadata object
    """
    # Path setup
    tax_export_path = os.path.join(base_dir, 'taxonomy.qzv')
    barplot_export_path = os.path.join(base_dir, 'taxonomy_barplot.qzv')

    # Load metadata
    taxonomy_metadata = qiime2.Metadata.from_artifact(taxonomy_analysis.classification)

    # Create taxonomy visualization
    taxonomy_visualization = metadata.visualizers.tabulate(taxonomy_metadata)

    # Save taxonomy visualization
    taxonomy_visualization.visualization.save(tax_export_path)
    logging.info('Saved {} successfully'.format(tax_export_path))

    # Create and save barplot visualization
    taxonomy_barplot = taxa.visualizers.barplot(table=dada2_filtered_table,
                                                taxonomy=taxonomy_analysis.classification,
                                                metadata=metadata_object)
    taxonomy_barplot.visualization.save(barplot_export_path)
    logging.info('Saved {} successfully'.format(barplot_export_path))

    return taxonomy_metadata


def run_diversity_metrics(base_dir, dada2_filtered_table, phylo_rooted_tree, metadata_object, sampling_depth=None):
    """
    :param base_dir: Main working directory filepath
    :param dada2_filtered_table: 
    :param phylo_rooted_tree: 
    :param metadata_object: 
    :param sampling_depth: 
    :return: qiime2 diversity core metrics object
    """
    logging.info('Attempting to calculate diversity metrics')

    # Set sampling_depth to 10% of the maximum if no value is provided. Should rework how this value is generated.
    if sampling_depth is None:
        sampling_depth = int(calculate_maximum_depth(dada2_filtered_table) * 0.1)

    # Path setup
    bray_curtis_path = os.path.join(base_dir, 'bray_curtis_emperor.qzv')
    jaccard_emperor_path = os.path.join(base_dir, 'jaccard_emperor.qzv')
    unweighted_unifrac_emperor_path = os.path.join(base_dir, 'unweighted_unifrac_emperor.qzv')
    weighted_unifrac_emperor_path = os.path.join(base_dir, 'weighted_unifrac_emperor.qzv')

    faith_visualization_path = os.path.join(base_dir, 'faith-pd-group-significance.qzv')
    evenness_visualization_path = os.path.join(base_dir, 'evenness-group-significance.qzv')
    beta_visualization_path = os.path.join(base_dir, 'unweighted-unifrac-sample-type-significance.qzv')

    # Retrieve diversity metrics
    diversity_metrics = diversity.pipelines.core_metrics_phylogenetic(table=dada2_filtered_table,
                                                                      phylogeny=phylo_rooted_tree.rooted_tree,
                                                                      sampling_depth=sampling_depth,
                                                                      metadata=metadata_object)

    # Save
    diversity_metrics.bray_curtis_emperor.save(bray_curtis_path)
    logging.info('Saved {} successfully'.format(bray_curtis_path))

    diversity_metrics.jaccard_emperor.save(jaccard_emperor_path)
    logging.info('Saved {} successfully'.format(jaccard_emperor_path))

    diversity_metrics.unweighted_unifrac_emperor.save(unweighted_unifrac_emperor_path)
    logging.info('Saved {} successfully'.format(unweighted_unifrac_emperor_path))

    diversity_metrics.weighted_unifrac_emperor.save(weighted_unifrac_emperor_path)
    logging.info('Saved {} successfully'.format(weighted_unifrac_emperor_path))

    # Alpha group significance
    alpha_group_faith = diversity.visualizers.alpha_group_significance(
        alpha_diversity=diversity_metrics.faith_pd_vector,
        metadata=metadata_object)

    alpha_group_evenness = diversity.visualizers.alpha_group_significance(
        alpha_diversity=diversity_metrics.evenness_vector,
        metadata=metadata_object)

    # Save
    alpha_group_faith.visualization.save(faith_visualization_path)
    logging.info('Saved {} successfully'.format(faith_visualization_path))
    alpha_group_evenness.visualization.save(evenness_visualization_path)
    logging.info('Saved {} successfully'.format(evenness_visualization_path))

    # Beta group significance
    try:
        beta_group = diversity.visualizers.beta_group_significance(
            distance_matrix=diversity_metrics.unweighted_unifrac_distance_matrix,
            metadata=metadata_object.get_category('sample sub-sub-type'),
            pairwise=True)
        beta_group.visualization.save(beta_visualization_path)
    except:
        logging.info('Could not calculate beta group significance with metadata feature Sample_Type\n')

    return diversity_metrics


# TODO: Implement this function to allow for training
def train_feature_classifier(base_dir, otu_filepath, reference_taxonomy_filepath, f_primer=None, r_primer=None):
    """
    Trains a Naive Bayes classifier based on a reference database/taxonomy
    :param base_dir: Main working directory filepath
    :param otu_filepath: File path to reference OTU .qza file
    :param reference_taxonomy_filepath: File path to reference taxonomy .qza file
    :param f_primer: String containing forward primer sequence. Default V3-V4 regions.
    :param r_primer: String containing reverse primer sequence Default V3-V4 regions.
    :return: Returns the trained feature classifier
    """
    if f_primer is None and r_primer is None:
        f_primer = 'CCTACGGGNGGCWGCAG'  # V3-V4
        r_primer = 'GACTACHVGGGTATCTAATCC'  # V3-V4

    # Path setup
    ref_seqs_filepath = os.path.join(base_dir, 'ref-seqs.qza')

    otus = qiime2.Artifact.load(otu_filepath)
    ref_taxonomy = qiime2.Artifact.load(reference_taxonomy_filepath)
    reference_seqs = feature_classifier.methods.extract_reads(sequences=otus,
                                                              f_primer=f_primer,
                                                              r_primer=r_primer)
    reference_seqs.reads.save(ref_seqs_filepath)
    naive_bayes_classifier = feature_classifier.methods.fit_classifier_naive_bayes(reference_reads=reference_seqs.reads,
                                                                                   reference_taxonomy=ref_taxonomy)

    return naive_bayes_classifier


# TODO: Implement these features https://docs.qiime2.org/2017.12/tutorials/longitudinal/ for longitudinal data
def longitudinal_comparison(base_dir):
    pass


def merge_feature_tables(base_dir):
    pass


def run_qc_pipeline(base_dir, data_artifact_path, sample_metadata_path):
    """
    1. Loads qiime2 generated data artifact and sample metadata file into a qiime2 artifact
    2. Generates a visualization of pre-filtering sequence quality

    :param base_dir: Main working directory filepath
    :param data_artifact_path:
    :param sample_metadata_path:
    """
    # Load seed objects
    data_artifact = load_data_artifact(data_artifact_path)
    metadata_object = load_sample_metadata(sample_metadata_path)

    # Visualize metadata
    metadata_viz = visualize_metadata(base_dir=base_dir, metadata_object=metadata_object)

    # Demux
    demux_viz = visualize_demux(base_dir=base_dir, data_artifact=data_artifact)


def read_metadata_df(sample_metadata_path):
    df = pd.read_csv(sample_metadata_path, delimiter='\t')
    return df


def validate_sample_id(sample_id):
    if not sample_id.endswith('_00'):
        sample_id += '_00'
        logging.debug('Sample ID in metadata not correctly named --> corrected to: {}'.format(sample_id))
    return sample_id


def write_new_metadata(df, sample_metadata_path):
    new_metadata_path = os.path.join(os.path.dirname(sample_metadata_path),
                                     os.path.basename(sample_metadata_path).replace('.tsv','_Validated.tsv'))
    df.to_csv(new_metadata_path, sep='\t', index=None)
    return new_metadata_path


def validate_metadata(base_dir, sample_metadata_path):
    df = read_metadata_df(sample_metadata_path)
    df['#SampleID'] = df['#SampleID'].apply(validate_sample_id)  # Assumption that first column is the SampleID column
    new_metadata_path = write_new_metadata(df, sample_metadata_path)
    return new_metadata_path


# Path setup

In [125]:
metadata_merged = '/home/dussaultf/Documents/qiime2/171206_180131_Merged/16s_samples_meta-data_Ver1_Validated_MERGED_v2.tsv'

table1_artifact_path = '/home/dussaultf/Documents/qiime2/171206_180131_Merged/Run1/output_v2/qiime2/table-dada2.qza'
table2_artifact_path = '/home/dussaultf/Documents/qiime2/171206_180131_Merged/Run2/output_v2/qiime2/table-dada2.qza'

repseqs1_artifact_path = '/home/dussaultf/Documents/qiime2/171206_180131_Merged/Run1/output_v2/qiime2/rep-seqs-dada2.qza'
repseqs2_artifact_path = '/home/dussaultf/Documents/qiime2/171206_180131_Merged/Run2/output_v2/qiime2/rep-seqs-dada2.qza'

pork_samples = load_sample_metadata('/home/dussaultf/Documents/qiime2/171206_180131_Merged/pork_sampleIDs.tsv')
beef_samples = load_sample_metadata('/home/dussaultf/Documents/qiime2/171206_180131_Merged/beef_sampleIDs.tsv')
sprouts_samples = load_sample_metadata('/home/dussaultf/Documents/qiime2/171206_180131_Merged/sprout_sampleIDs.tsv')
veal_samples = load_sample_metadata('/home/dussaultf/Documents/qiime2/171206_180131_Merged/veal_sampleIDs.tsv')

pork_dir = '/home/dussaultf/Documents/qiime2/171206_180131_Merged/Merged_output/Pork_analysis'
beef_dir = '/home/dussaultf/Documents/qiime2/171206_180131_Merged/Merged_output/Beef_analysis'
sprouts_dir = '/home/dussaultf/Documents/qiime2/171206_180131_Merged/Merged_output/Sprouts_analysis'
veal_dir = '/home/dussaultf/Documents/qiime2/171206_180131_Merged/Merged_output/Veal_analysis'

base_dir = '/home/dussaultf/Documents/qiime2/171206_180131_Merged/Merged_output'
classifier_artifact_path = '/home/dussaultf/PycharmProjects/AmpliconPipeline/classifiers/99_V3V4_Silva_naive_bayes_classifier.qza'

In [126]:
metadata_object = load_sample_metadata(metadata_merged)
dada2_filtered_table = merge_run_tables(table1_artifact_path, table2_artifact_path)
dada2_filtered_rep_seqs = merge_run_repseqs(repseqs1_artifact_path, repseqs2_artifact_path)

In [94]:
dada2_filtered_rep_seqs

<artifact: FeatureData[Sequence] uuid: ab797eec-b8a1-490d-870b-e54bc4adca60>

# All samples feature table filtering

In [127]:
sprouts_table = feature_table.actions.filter_samples(
    table = dada2_filtered_table,
    metadata=sprouts_samples,
).filtered_table

pork_table = feature_table.actions.filter_samples(
    table = dada2_filtered_table,
    metadata=pork_samples,
).filtered_table

veal_table = feature_table.actions.filter_samples(
    table = dada2_filtered_table,
    metadata=veal_samples,
).filtered_table

beef_table = feature_table.actions.filter_samples(
    table = dada2_filtered_table,
    metadata=beef_samples,
).filtered_table

# All samples rep seqs filtering

In [128]:
sprouts_rep_seqs = feature_table.actions.filter_seqs(
    data=dada2_filtered_rep_seqs,
    metadata=sprouts_samples,
    exclude_ids=True
).filtered_data

beef_rep_seqs = feature_table.actions.filter_seqs(
    data=dada2_filtered_rep_seqs,
    metadata=beef_samples,
    exclude_ids=True
).filtered_data

pork_rep_seqs = feature_table.actions.filter_seqs(
    data=dada2_filtered_rep_seqs,
    metadata=pork_samples,
    exclude_ids=True
).filtered_data

veal_rep_seqs = feature_table.actions.filter_seqs(
    data=dada2_filtered_rep_seqs,
    metadata=veal_samples,
    exclude_ids=True
).filtered_data


# Sprouts

In [129]:
# Mask and alignment
(seq_mask, seq_alignment) = seq_alignment_mask(base_dir=sprouts_dir,
                                               dada2_filtered_rep_seqs=sprouts_rep_seqs)

# Phylogenetic tree
(phylo_unrooted_tree, phylo_rooted_tree) = phylo_tree(base_dir=sprouts_dir, seq_mask=seq_mask)

# Load classifier
classifier = load_classifier_artifact(classifier_artifact_path=classifier_artifact_path)


# Produce rarefaction visualization
alpha_rarefaction_viz = alpha_rarefaction_visualization(base_dir=sprouts_dir,
                                                        dada2_filtered_table=sprouts_table)

# Run taxonomic analysis
taxonomy_analysis = classify_taxonomy(base_dir=sprouts_dir,
                                      dada2_filtered_rep_seqs=sprouts_rep_seqs,
                                      classifier=classifier)

# Visualize taxonomy
taxonomy_metadata = visualize_taxonomy(base_dir=sprouts_dir,
                                       metadata_object=metadata_object,
                                       taxonomy_analysis=taxonomy_analysis,
                                       dada2_filtered_table=sprouts_table)

# Alpha and beta diversity
# TODO: requires metadata object with some sort of sample information (sample type). Need to handle this gracefully.
diversity_metrics = run_diversity_metrics(base_dir=sprouts_dir,
                                          dada2_filtered_table=sprouts_table,
                                          phylo_rooted_tree=phylo_rooted_tree,
                                          metadata_object=metadata_object)


Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: mafft --preservecase --inputorder --thread 12 /tmp/qiime2-archive-q1me0rcw/35fa1644-154e-422f-a6fe-2f8cf6b1c3f6/data/dna-sequences.fasta

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: FastTree -quote -nt /tmp/qiime2-archive-p0r0tp2x/6b75dac8-37f3-4940-9e32-adfa12a0b068/data/aligned-dna-sequences.fasta



/home/dussaultf/miniconda3/envs/qiime2-2017.11/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)
/home/dussaultf/miniconda3/envs/qiime2-2017.11/lib/python3.5/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:111: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.04452750622693573 and the largest is 3.008691911778935.
  RuntimeWarning
/home/dussaultf/miniconda3/envs/qiime2-2017.11/lib/python3.5/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:111: RuntimeWarning: The result contains negative 

# Beef

In [130]:
# Mask and alignment
(seq_mask, seq_alignment) = seq_alignment_mask(base_dir=beef_dir,
                                               dada2_filtered_rep_seqs=beef_rep_seqs)

# Phylogenetic tree
(phylo_unrooted_tree, phylo_rooted_tree) = phylo_tree(base_dir=beef_dir, seq_mask=seq_mask)

# Load classifier
classifier = load_classifier_artifact(classifier_artifact_path=classifier_artifact_path)


# Produce rarefaction visualization
alpha_rarefaction_viz = alpha_rarefaction_visualization(base_dir=beef_dir,
                                                        dada2_filtered_table=beef_table)

# Run taxonomic analysis
taxonomy_analysis = classify_taxonomy(base_dir=beef_dir,
                                      dada2_filtered_rep_seqs=beef_rep_seqs,
                                      classifier=classifier) 

# Visualize taxonomy
taxonomy_metadata = visualize_taxonomy(base_dir=beef_dir,
                                       metadata_object=metadata_object,
                                       taxonomy_analysis=taxonomy_analysis,
                                       dada2_filtered_table=beef_table)

# Alpha and beta diversity
# TODO: requires metadata object with some sort of sample information (sample type). Need to handle this gracefully.
diversity_metrics = run_diversity_metrics(base_dir=beef_dir,
                                          dada2_filtered_table=beef_table,
                                          phylo_rooted_tree=phylo_rooted_tree,
                                          metadata_object=metadata_object)




Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: mafft --preservecase --inputorder --thread 12 /tmp/qiime2-archive-ngz_rz1v/92392f83-30fe-4d8b-8843-a8900fb084c8/data/dna-sequences.fasta

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: FastTree -quote -nt /tmp/qiime2-archive-ro9i3m0y/ea3286e6-a1bf-49ff-9c99-1631e8e603ef/data/aligned-dna-sequences.fasta



/home/dussaultf/miniconda3/envs/qiime2-2017.11/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)
/home/dussaultf/miniconda3/envs/qiime2-2017.11/lib/python3.5/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:111: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.06112930171166014 and the largest is 2.474650432528974.
  RuntimeWarning
/home/dussaultf/miniconda3/envs/qiime2-2017.11/lib/python3.5/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:111: RuntimeWarning: The result contains negative 

# Veal

In [131]:
# Mask and alignment
(seq_mask, seq_alignment) = seq_alignment_mask(base_dir=veal_dir,
                                               dada2_filtered_rep_seqs=veal_rep_seqs)

# Phylogenetic tree
(phylo_unrooted_tree, phylo_rooted_tree) = phylo_tree(base_dir=veal_dir, seq_mask=seq_mask)

# Load classifier
classifier = load_classifier_artifact(classifier_artifact_path=classifier_artifact_path)


# Produce rarefaction visualization
alpha_rarefaction_viz = alpha_rarefaction_visualization(base_dir=veal_dir,
                                                        dada2_filtered_table=veal_table)

# Run taxonomic analysis
taxonomy_analysis = classify_taxonomy(base_dir=veal_dir,
                                      dada2_filtered_rep_seqs=veal_rep_seqs,
                                      classifier=classifier)

# Visualize taxonomy
taxonomy_metadata = visualize_taxonomy(base_dir=veal_dir,
                                       metadata_object=metadata_object,
                                       taxonomy_analysis=taxonomy_analysis,
                                       dada2_filtered_table=veal_table)

# Alpha and beta diversity
# TODO: requires metadata object with some sort of sample information (sample type). Need to handle this gracefully.
diversity_metrics = run_diversity_metrics(base_dir=veal_dir,
                                          dada2_filtered_table=veal_table,
                                          phylo_rooted_tree=phylo_rooted_tree,
                                          metadata_object=metadata_object)


Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: mafft --preservecase --inputorder --thread 12 /tmp/qiime2-archive-rwgi5o7p/67d2bb6a-b60b-4359-a5c9-fa74bcb07e84/data/dna-sequences.fasta

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: FastTree -quote -nt /tmp/qiime2-archive-u7gt5idr/076b1e98-e5df-4630-9483-5f71f18cb054/data/aligned-dna-sequences.fasta



/home/dussaultf/miniconda3/envs/qiime2-2017.11/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)
/home/dussaultf/miniconda3/envs/qiime2-2017.11/lib/python3.5/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:111: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.0717162811812427 and the largest is 4.142900050977926.
  RuntimeWarning
/home/dussaultf/miniconda3/envs/qiime2-2017.11/lib/python3.5/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:111: RuntimeWarning: The result contains negative e

# Pork

In [132]:
# Mask and alignment
(seq_mask, seq_alignment) = seq_alignment_mask(base_dir=pork_dir,
                                               dada2_filtered_rep_seqs=pork_rep_seqs)

# Phylogenetic tree
(phylo_unrooted_tree, phylo_rooted_tree) = phylo_tree(base_dir=pork_dir, seq_mask=seq_mask)

# Load classifier
classifier = load_classifier_artifact(classifier_artifact_path=classifier_artifact_path)


# Produce rarefaction visualization
alpha_rarefaction_viz = alpha_rarefaction_visualization(base_dir=pork_dir,
                                                        dada2_filtered_table=pork_table)

# Run taxonomic analysis
taxonomy_analysis = classify_taxonomy(base_dir=pork_dir,
                                      dada2_filtered_rep_seqs=pork_rep_seqs,
                                      classifier=classifier)

# Visualize taxonomy
taxonomy_metadata = visualize_taxonomy(base_dir=pork_dir,
                                       metadata_object=metadata_object,
                                       taxonomy_analysis=taxonomy_analysis,
                                       dada2_filtered_table=pork_table)

# Alpha and beta diversity
# TODO: requires metadata object with some sort of sample information (sample type). Need to handle this gracefully.
diversity_metrics = run_diversity_metrics(base_dir=pork_dir,
                                          dada2_filtered_table=pork_table,
                                          phylo_rooted_tree=phylo_rooted_tree,
                                          metadata_object=metadata_object)


Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: mafft --preservecase --inputorder --thread 12 /tmp/qiime2-archive-9w9vk14o/4cd90d7e-f41f-4f76-881e-1b089813071c/data/dna-sequences.fasta

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: FastTree -quote -nt /tmp/qiime2-archive-w3jha00b/c389b321-8dd6-4eac-ba13-edca44a9d510/data/aligned-dna-sequences.fasta



/home/dussaultf/miniconda3/envs/qiime2-2017.11/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)
/home/dussaultf/miniconda3/envs/qiime2-2017.11/lib/python3.5/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:111: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.015032308739108628 and the largest is 2.7512131369752293.
  RuntimeWarning
/home/dussaultf/miniconda3/envs/qiime2-2017.11/lib/python3.5/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:111: RuntimeWarning: The result contains negativ

# Read statistics

In [119]:
run1_table_stats = load_data_artifact('/home/dussaultf/Documents/qiime2/171206_180131_Merged/Run1/output/qiime2/table-dada2.qza')
run2_table_stats = load_data_artifact('/home/dussaultf/Documents/qiime2/171206_180131_Merged/Run2/output/qiime2/table-dada2.qza')

In [122]:
df = run1_table_stats.view(pd.DataFrame)
df = df.transpose()
df

,2017-SEQ-1110_00,2017-SEQ-1111_00,2017-SEQ-1112_00,2017-SEQ-1113_00,2017-SEQ-1114_00,2017-SEQ-1115_00,2017-SEQ-1116_00,2017-SEQ-1117_00,2017-SEQ-1118_00,2017-SEQ-1119_00,...,2017-SEQ-1148_00,2017-SEQ-1149_00,2017-SEQ-1150_00,2017-SEQ-1151_00,2017-SEQ-1152_00,2017-SEQ-1153_00,2017-SEQ-1154_00,2017-SEQ-1155_00,2017-SEQ-1156_00,2017-SEQ-1157_00
a01934271f6c9eec177cf854f8ba7609,0.0,0.0,0.0,0.0,75.0,144.0,0.0,114.0,0.0,0.0,...,16003.0,203.0,15821.0,25769.0,3425.0,27734.0,15356.0,1081.0,28306.0,21905.0
5b69d232647d604a89cc1f2e294274be,0.0,0.0,0.0,0.0,105.0,0.0,0.0,0.0,107.0,0.0,...,13327.0,199.0,13599.0,21088.0,2929.0,23198.0,12654.0,859.0,24106.0,17892.0
119f3c2a07af32e3db3800f7f60afb8a,0.0,133.0,99.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9852.0,120.0,8869.0,15575.0,2098.0,16938.0,9280.0,606.0,16937.0,13132.0
2b1318790ea94462200a7e748084a2c4,0.0,0.0,0.0,0.0,0.0,142.0,0.0,179.0,0.0,0.0,...,22735.0,147.0,24496.0,40112.0,4760.0,36329.0,22002.0,1390.0,37333.0,30197.0
fd6f77b06c9335ece308751b7bc10207,0.0,72.0,128.0,0.0,84.0,0.0,0.0,0.0,0.0,0.0,...,18908.0,143.0,21482.0,33295.0,3888.0,30230.0,17779.0,1093.0,32088.0,24737.0
6fe03ce4367d9ed5e63a4e172ac4b29c,0.0,0.0,0.0,0.0,0.0,76.0,0.0,0.0,0.0,0.0,...,13932.0,90.0,7307.0,11023.0,3061.0,22895.0,14030.0,759.0,24006.0,19292.0
e1941c3ef0f7f090131f142ac296b374,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,13830.0,95.0,13759.0,23942.0,2708.0,21507.0,13079.0,840.0,22241.0,18700.0
15654017c73c8bb55fd8761ae05f6dd0,77.0,57.0,0.0,0.0,58.0,0.0,0.0,0.0,0.0,0.0,...,1253.0,1371.0,23362.0,1064.0,0.0,0.0,0.0,0.0,0.0,0.0
e0b73158da1e991e0753e0c33ae346e7,48080.0,18546.0,844.0,33985.0,13269.0,1051.0,1068.0,331.0,0.0,1121.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
204a6d0222528c1f18e2bbdf4d22f8d2,0.0,0.0,0.0,0.0,0.0,0.0,36.0,0.0,0.0,0.0,...,936.0,1208.0,20499.0,792.0,0.0,0.0,0.0,0.0,0.0,0.0
